<a href="https://colab.research.google.com/github/ugurcun/Saatlik-Enerji-Talebi-Uretimi-ve-Hava-Durumu/blob/main/Saatlik_Enerji_Talebi_%C3%9Cretimi_ve_Hava_Durumu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# 1. Veriyi yükleme
energy_file = "energy_dataset.csv"
weather_file = "weather_features.csv"
energy_df = pd.read_csv(energy_file)
weather_df = pd.read_csv(weather_file)

# Çok fazla eksik veri içeren sütunları kaldırma
energy_df.drop(columns=['generation hydro pumped storage aggregated', 'forecast wind offshore eday ahead'], inplace=True)

# Eksik verileri sütun medyanı ile doldurma
numeric_columns = energy_df.select_dtypes(include=np.number).columns
energy_df[numeric_columns] = energy_df[numeric_columns].fillna(energy_df[numeric_columns].median())

# 2. Aykırı değerleri IQR yöntemine göre temizleme
# Sadece sayısal sütunları seçin
numeric_energy_df = energy_df.select_dtypes(include=np.number)

Q1 = numeric_energy_df.quantile(0.25)
Q3 = numeric_energy_df.quantile(0.75)
IQR = Q3 - Q1

# Alt ve üst sınırları belirleme
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Sınırların dışında kalan değerleri alt veya üst sınıra çekme
# Sadece sayısal sütunları güncelleyin
energy_df[numeric_columns] = numeric_energy_df.clip(lower=lower_bound, upper=upper_bound, axis=1)


# 3. Hata metriğini belirleme (RMSE kullanılıyor)
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# 4. Veriyi Train-Validation-Test olarak bölme
y = energy_df['total load actual']  # Örnek hedef değişken
X = energy_df.drop(columns=['total load actual', 'time'])  # Zaman sütunu çıkarılıyor

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 5. Farklı modelleri deneyerek en iyisini seçme
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
error = rmse(y_val, y_pred)

print(f"Validation RMSE: {error}")

# Modeli test setinde değerlendirme
y_test_pred = model.predict(X_test)
test_error = rmse(y_test, y_test_pred)
print(f"Test RMSE: {test_error}")

# Temizlenmiş veriyi kaydetme
energy_df.to_csv("cleaned_energy_dataset.csv", index=False)
print("Tüm veri işleme adımları tamamlandı ve model değerlendirildi.")

Validation RMSE: 464.8710565269706
Test RMSE: 420.44530902707277
Tüm veri işleme adımları tamamlandı ve model değerlendirildi.
